<a href="https://colab.research.google.com/github/Hbasgol/ibm_models/blob/master/IBM_Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

###################### modules/libraries #######################################
#
# import os -> to see the path of folders and libraries
#
# import time -> to learn the timing of each step of expectation-maximization
#
# from google.colab import drive -> to connect google drive
#
# import codecs -> to save Turkish characters without a problem
#
# import json -> to save dictionaries structured for representing tables as
#   .json documents, json.dump is used for writing and json.loads is used for
#   reading .json strings as dictionaries
#
# from itertools import product
# from itertools import permutations -> to find possible alignments between
#   English and Turkish phrases that structure corresponding sentences
#
# import numpy as np -> for simple mathematical operations such as taking absolute
#   of a number with np.abs or summing values in a list with np.sum
#
# import unicodedata -> for removing Turkish characters in the corpus
#
################################################################################
import os
import time
from google.colab import drive
import codecs
import json
from itertools import permutations
from itertools import product
import numpy as np
import unicodedata
################################################################################


###################### connectcolab ############################################
# the function connectcolab is used to receive Google Drive documents and
#   determine the path the files are written to and read from.
################################################################################
def connectcolab():
  drive.mount('/content/drive', force_remount=True)
  os.chdir("/content/drive/My Drive/Colab Notebooks/Machine Translation/")
path = "/content/drive/My Drive/Colab Notebooks/Machine Translation/IBM1-tables/"
################################################################################


###################### tokenization ############################################
# takes sentencelist such as [["first sentence"], ["second sentence"], ["third sentence"] ...] 
#   two operators named op1 and op2
#   op1 is used to determine the translation direction whether English to Turkish
#    or Turkish to English, because IBM 1 and IBM 2 should be run bi-directional
#    to get word-alignments for phrase extraction in phrase-based translation
#   op2 is used to determine the language of sentences because Turkish characters
#    are removed due to the inconsistency in the corpus. For removing Turkish
#    characters, a helper function named rm_turkish is used
# The function
#  returns [["first", "sentence"], ["second", "sentence"], ["third", "sentence"] ...]
#  or
#  returns  [["NULL", "first", "sentence"], ["NULL", "second", "sentence"] ...]
################################################################################

def tokenization(sentencelist, op1, op2):
  if op1 == "t2e":
    if op2 == "t":
      return [["NULL"] + [*map(rm_turkish, i.split(" "))] for i in sentencelist]
    if op2 == "e":
      return [i.split(" ") for i in sentencelist]
  if op1 == "e2t":
    if op2 == "t":
      return [[*map(rm_turkish, i.split(" "))] for i in sentencelist]
    if op2 == "e":
      return [["NULL"] + i.split(" ") for i in sentencelist]
    
################################################################################


#################### rm_turkish ################################################
# the function takes a word that is a string data type and change Turkish
# characters into English counterparts.
#
# takes uçuyorum -> returns ucuyorum
# şenlik -> senlik
#
# to remove the ambiguity of the corpus in terms of Turkish sentences
#
# the function is used in another function named tokenization
################################################################################

def rm_turkish(word):
  normalized = unicodedata.normalize('NFD', word)
  word = "".join([c for c in normalized if not unicodedata.combining(c)])
  return word

################################################################################


###################### get_words ###############################################
# get_words takes two arguments: english_list_train as target turkish_list_train as
#    source sentence, or vice versa, which does not affect code
#    such as [["first", "sentence"], ["second", "sentence"], ["third", "sentence"], ...]
#
# get_words returns unique words as two arguments 
#    such as [["first"], ["sentence"], ["second"], ["third"]]
################################################################################

def get_words(BU_sentences):
  wordset=set()
  for sentence in BU_sentences:
    for word in sentence:
      wordset.add(word)
  return list(wordset)

################################################################################


############################## get_t_table #####################################
# the function takes three arguments
#    english_list_train, turkish_list_train, turkish_tk_word)
#    english_list_train: english sentences such as
#       [["first", "sentence"], ["second", "sentence"], ["third", "sentence"], ...]
#    turkish_list_train: turkish sentences, respectively
#    turkish_tk_word: turkish words such as [["birinci"], ["cümle"], ["ikinci"], ["üçüncü"]]
#       for determining the values of translation table uniformly
#
# the function, throughout the execution, creates four tables as dictionaries
#    t_table, count_tables, s_totals, total_tables
#    normally, algorithm loops on all english and turkish words, however,
#    for memory issues, sentence matches are considered here
#    all respective values are added respective dictionaries
#    -t_table is initialized uniformly with val = 1/len(turkish_tk_word)
#    -count_tables, s_totals, total_tables are set to 0
#
# the function returns one arguments,
#    tablelist: which is a list involving four dictionaries:
#    t_table, count_tables, s_totals, total_tables
#
#    t_table is a dictionary like
#    t_table = {"Hello":
#                {Book: 0.32}
#                {Furniture: 0.12} ...}
#    0.32 by which is received t_table["Hello"]["Book"] count_tables is same
#    s_totals is like
#    s_totals = {"Hello": 0.218,
#                "Furniture": 0.45 ...} total tables is same
################################################################################

def get_t_table(english_list_train, turkish_list_train, turkish_tk_word):
  #empty dictionaries are genereated
  t_table, count_tables, s_totals, total_tables = {}, {}, {}, {}
  val = 1/len(turkish_tk_word)
  for index_e, e_sentence in enumerate(english_list_train): # for each english sentence
    f_sentence = turkish_list_train[index_e] # determine foreign sentence with respective index
    for j, e_word in enumerate(e_sentence):
      if e_word not in count_tables:
        count_tables[e_word]={}
        t_table[e_word]={}
      s_totals[e_word]=0
      for i, f_word in enumerate(f_sentence):
        total_tables[f_word]=0
        count_tables[e_word].update({f_word: 0})
        t_table[e_word].update({f_word: val})
  tablelist = [t_table, count_tables, s_totals, total_tables] # create list
  return tablelist

################################################################################


############################## save/read_t_table ###############################
# save_t_table takes a specific dictionary, involving t_tables and writes it to
#    the disk as a .json file with the name of "IBM1-t_table.json"
# read_t_table does not take an argument, but reads "IBM1-t_table.json" as a
#    dictionary for further use
#
# these functions have an operator argument named op that takes the
#    direction of translation, which can be either "t2e" or "e2t"
################################################################################

def save_t_table(t_tables, op):
  with codecs.open(path+"IBM1-t_table"+op+".json", 'w', 'utf-8') as f:
    f.write(json.dumps(t_tables, indent=2, ensure_ascii=False))
    
def read_t_table(op):
  with codecs.open(path+"IBM1-t_table"+op+".json") as f:
    return json.loads(f.read())
  
################################################################################


############################## to_zero #########################################
# the function takes two arguments structured as nested dictionaries
#   count_tables, total_tables
#
# it changes the values of count_tables, total_tables to be 0, 
# 	which is a step of expectation_maximization.
#
# Then, it returns these tables as a list:
#   [count_tables, total_tables] 
################################################################################
def to_zero(count_tables, total_tables):
  for e_word, val in count_tables.items():
    for f_word, value in val.items():
      total_tables[f_word]=0
      count_tables[e_word][f_word]=0
  return [count_tables, total_tables]
################################################################################


############################### get_sample #####################################
# the functiont takes one argument, n_sample, which determines the number of 
#   sentences which will be using in the function. The n_sample of sentences
#   are then separated into test and train sets with a ratio of 1/9
# the function returns english_train, turkish_train, english_test, turkish_test, 
#   english_word, turkish_word
#   english_train: english sentences the model to be trained as a list of list
#   turkish_train: turkish sentences the model to be trained as a list of list
#   english_test: english sentences the model to be tested as a list of list
#   english_test: english sentences the model to be tested as a list of list
################################################################################
def get_sample(sample, t):
  connectcolab()
    
  english_list = []
  with open("/content/drive/My Drive/Colab Notebooks/Machine Translation/corpus/english.txt", "r") as english:
    for cnt, line in enumerate(english):
      english_list.append(line.rstrip())

  turkish_list = []
  with open("/content/drive/My Drive/Colab Notebooks/Machine Translation/corpus/turkish.txt", "r") as turkish:
    for cnt, line in enumerate(turkish):
      turkish_list.append(line.rstrip())
  
  english_list=english_list[:sample]
  turkish_list=turkish_list[:sample]
  
  rtrain = int(sample*0.9)

  english_list = tokenization(english_list, t, "e")
  turkish_list = tokenization(turkish_list, t, "t")
  english_word = get_words(english_list)
  turkish_word = get_words(turkish_list)
  english_train, turkish_train = english_list[:rtrain], turkish_list[:rtrain]
  english_test, turkish_test = english_list[rtrain:], turkish_list[rtrain:]
  print("sentences for training have been processed")
  return english_train, turkish_train, english_test, turkish_test, english_word, turkish_word
################################################################################


############################ perplexity ########################################
# The function takes four arguments:
#   english_train: English sentences
#   turkish_train: Turkish sentences
#   t_tables: translation probability table, word-based
#   alignments: alignments
#
# It returns perplexity, which is a float number representing how well the
#   probabilities that have been found are suited for the data
#   In each iteration of the expectation maximization algorithm, perplexity
#   reduces. If it does not change through iterations, this means that 
#   the algorithm converged.
#
# This function does not compute the whole perplexity formula, because it is
#   computationally inefficient, rather than computing the translation probability
#   of a sentence, it looks argmax for each word. Although the computation
#   is not same, it gives an idea regarding the convergence of the algorithm
################################################################################

def perplexity(english_train, turkish_train, t_tables):
  s=0
  for index_e, e_sentence in enumerate(english_train):
    pmax=1
    f_sentence = turkish_train[index_e]
    le = len(e_sentence)
    lf = len(f_sentence)
    s+=np.log(max(t_tables[e_word][f_word] \
			for e_word in e_sentence  \
			for f_word in f_sentence))
  print("perplexity: {}".format(-s))
  return -s
  
################################################################################

###################### expectation_maximization ################################
# The function takes english_word, turkish_word, english_train, turkish_train, tablelist, 
#   english_word: a list of english words in the corpus
#   turkish_word: a list of turkish words in the corpus
#   english_train: english sentences the model to be trained as a list of list
#   turkish_train: turkish sentences the model to be trained as a list of list
#   tablelist: is a list of dictionaries created by get_t_table
#     [t_table, count_tables, s_totals, total_tables]
#
# The function returns translation probabilities named t_table
################################################################################

def expectation_maximization(english_word, turkish_word, english_train, turkish_train, tablelist, op):
  [t_tables, count_tables, s_totals, total_tables] = tablelist
  count = 1                                      # to count how many steps taken
  epsilon = 1                                    # to determine convergence
  lastval = 0
  while True:
    print("✖ step {}".format(count))
    start = time.time()
    if count > 1:
      [count_tables, total_tables] = to_zero(count_tables, total_tables)
    for index_e, e_sentence in enumerate(english_train):
      f_sentence = turkish_train[index_e]
      for e_word in e_sentence:
        s_totals[e_word]=0
        for f_word in f_sentence:
          s_totals[e_word]+=t_tables[e_word][f_word]
      for e_word in e_sentence:
        for f_word in f_sentence:
          count_tables[e_word][f_word]+=(t_tables[e_word][f_word]/s_totals[e_word])
          total_tables[f_word]+=(t_tables[e_word][f_word]/s_totals[e_word])
    for e_word, val in t_tables.items():
      for f_word, value in val.items():
        t_tables[e_word][f_word]=(count_tables[e_word][f_word]/total_tables[f_word])
    table_rv, t_tables = normalize(t_tables)
    s = perplexity(english_train, turkish_train, t_tables)
    print("difference in perplexity:", s-lastval)
    end = time.time()
    print("execution time: {}".format(end-start))
    if not count==1:
      if abs(lastval- s) < epsilon or s > lastval:
        print("Expectation Maximization Algorithm is Converged within {} Steps".format(count))
        save_t_table(t_tables, op)
        break
    lastval = s
    count+=1
  return t_tables, table_rv
  
################################################################################


################################ prob_al #######################################
# The function takes four arguments:
#   t_tables, alignments, t_sentence as a Turkish sentence
#   e_sentence as an English sentence, op an operator to determine the direction
#   of translation.
#
# The function finds all probable alignments between source and target sentences
#   and returns a normalized probability for each of them with a nested dictionary
# Additionally, it sums up probabilities of all probable alignments to find
#   the translation probability of source and target sentence
################################################################################

def prob_al(t_tables, t_sentence, e_sentence, op):
  t_sentence = tokenization([t_sentence], op, "t")[0]
  e_sentence = tokenization([e_sentence], op, "e")[0]
  if op=="t2e":         # table is t[e_word][f_word] or t[target][source]
    source = t_sentence # determine source and target sentences
    target = e_sentence
  else:
    source = e_sentence
    target = t_sentence

  print("source sentence:", source)
  print("target sentence:", target)

  al=[]
  for t in [" "]+target: # source can be not-aligned
    wal=[]
    for s in source:
      wal.append([s, t]) # probable alignments
    al.append(wal)

  ali={}
  ind=0

  for a in product(*al): # find sentence alignments
                                   # target should have one alignment
    if len(set([l[1] for l in a])) == len(target)+1: 
      p_a=1
      for l in a:
        if l[1]!=" ":
          # translation and alignment prob.
          if l[1] in t_tables and l[0] in t_tables[l[1]]:
            p_a*=t_tables[l[1]][l[0]]
            
          else:
            p_a*=0
      if p_a!=0:
        ali[ind]=[a, p_a]
    ind+=1

  p_sum=0
  for key in ali:
    p_sum+=ali[key][1] # sum over probabilities
    
  for key in ali:
    if p_sum!=0:
      ali[key][1]/=p_sum # normalization

  values = list(set([ali[key][1] for key in ali]))
  values.sort(reverse = True)
  
  sortedali = {}
  for val in values[:2]:
      for key in ali:
          if ali[key][1] == val:
              sortedali[key]=ali[key]
           

  return sortedali, p_sum
################################################################################
      

################################ prob_word #####################################
# The function takes three arguments
#   t_table as a t_table
#   word as a word in the corpus
#   number is the maximum n probable words for word
#
# The function returns a list of tuples
#   such that 
#   maxtuplist=[(ev, house, 0.5), 
#               (ev, home, 0.4), 
#               (ev, book, 0.001) ...]
################################################################################

def prob_word(t_table, word, n):
  tuplist=[]
  maxtuplist=[]
  for t_word, t_values in t_tafble.items():
    for s_word, s_values in t_values.items():
      if word == s_word:
        tuplist.append((s_word, t_word, s_values))
  values=sorted([i[2] for i in tuplist])[:n-1]
  for val in values:
    for tup in tuplist:
      if tup[2]==val:
        maxtuplist.append(tup)
  return maxtuplist
################################################################################

def table_rv(t_tables):
  rv_table={}
  for key, val in t_tables.items():
    for k2, v2 in val.items():
      if k2 not in rv_table:
        rv_table[k2]={}
      rv_table[k2][key]=v2
  return rv_table

def normalize(t_table):
  rv_table = table_rv(t_table)
  for f, e in rv_table.items():
    n = sum(list(e.values()))
    for ew in e:
      e[ew]/=n
  for e, f in t_table.items():
    for f_word in f:
      t_table[e][f_word]=rv_table[f_word][e]
  return rv_table, t_table

def train(n_sample, d, t):
  english_train, turkish_train, english_test, turkish_test, english_word, turkish_word = get_sample(n_sample, t)
  print("Number of English Sentence {}, Turkish Sentence {}".format(len(english_train), len(turkish_train)))
  print("Number of English Word {}, Turkish Word {}".format(len(english_word), len(turkish_word)))
  if t == "t2e":
    tablelist = get_t_table(english_train, turkish_train, turkish_word)
    t_tables, table_rv = expectation_maximization(english_word, turkish_word, english_train, turkish_train, tablelist, d)
  elif t == "e2t":
    tablelist = get_t_table(turkish_train, english_train, english_word)
    t_tables, table_rv = expectation_maximization(turkish_word, english_word, turkish_train, english_train, tablelist, d)
  return t_tables, table_rv

In [0]:
t_sentence = "Okula giderim"
e_sentence = "I go to the school"